In [3]:
import os, sys

from datetime import datetime, timedelta
import time

import requests, json
from web3 import Web3, HTTPProvider

from eth_utils import to_bytes
from eth_keys import keys
from eth_account import Account
from eth_account.messages import SignableMessage

In [61]:
w3 = Web3(HTTPProvider("https://1rpc.io/eth"))

In [62]:
source_token = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984" # UniSwap
destination_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT

source_amount = 10000000000000000000
slippage = int(0.20 * 10000) # Convert to bps

In [64]:
eth_wallet = Account.from_key("e5ecbcfc71ad24a2131953ef9a66e4b7ed8ca9a10ffd92104e106fab1f0af4f7")

In [65]:
# 0.  One fresh quote – DO NOT fetch another one before sending
quote  = requests.get(
    "https://api.0x.org/swap/permit2/quote",
    params={
        "chainId": 1,
        "sellToken": source_token,
        "buyToken":  destination_token,
        "sellAmount": source_amount,
        "taker": eth_wallet.address,          # wallet is your LocalAccount
    },
    headers={
        "0x-version" : "v2",
        "0x-api-key" : "50cbb531-4fda-4464-acb4-5e6b5bf0e2d6"
    },
    timeout=10,
).json()

In [66]:
ERC20_ABI: list[dict] = [
    {
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
]

erc20 = w3.eth.contract(address=Web3.to_checksum_address(source_token), abi=ERC20_ABI)

sell_amount   = int(quote["sellAmount"])
permit_amount = int(quote["permit2"]["eip712"]["message"]["permitted"]["amount"])
owner_balance = int(erc20.functions.balanceOf(eth_wallet.address).call())

print ("sellAmount        :", sell_amount)
print ("permit.amount     :", permit_amount)
print ("wallet UNI balance:", owner_balance)
print ()
print ("gas fee:", (1.0e-18) * int(quote['transaction']['gas']) * int(quote['transaction']['gasPrice']), "ETH")

sellAmount        : 10000000000000000000
permit.amount     : 10000000000000000000
wallet UNI balance: 97116026654249141437

gas fee: 0.00022402019106951203 ETH


In [67]:
# 1.  Sign **exactly** the 32‑byte digest in that quote
digest   = Web3.to_bytes(hexstr=quote["permit2"]["hash"])      # 32 bytes
priv     = keys.PrivateKey(eth_wallet.key)
sig_obj  = priv.sign_msg_hash(digest)                     # v is already 27/28

signature = sig_obj.to_bytes()                            # 65 bytes

assert len(signature) == 65

In [68]:
# 2.  Prove locally that the router will recover the right address

rec_addr = sig_obj.recover_public_key_from_msg_hash(digest).to_checksum_address()

print("wallet.address :", eth_wallet.address)
print("recovered       :", rec_addr)

assert rec_addr.lower() == eth_wallet.address.lower(), "still mismatched"

wallet.address : 0xa88e05Ce22569229B75b9EAE47934821Aee5C992
recovered       : 0xa88e05Ce22569229B75b9EAE47934821Aee5C992


In [69]:
# 3.  Append length + signature to the calldata

PLACEHOLDER = 32 + 65

orig_data = Web3.to_bytes(hexstr=quote["transaction"]["data"])

calldata = (
    orig_data[:-PLACEHOLDER] +
    len(signature).to_bytes(32, "big") +
    signature
)

In [70]:
# 4. Build, sign, send
tx = {
    "to":       Web3.to_checksum_address(quote["transaction"]["to"]),
    "data":     Web3.to_hex(calldata),
    "value":    int(quote["transaction"]["value"]),
    "gas":      int(quote["transaction"]["gas"]),
    "gasPrice": int(quote["transaction"]["gasPrice"]),
    "nonce":    w3.eth.get_transaction_count(eth_wallet.address),
    "chainId":  1,
}

tx_signed = eth_wallet.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(tx_signed.raw_transaction)

In [71]:
# 5. Check result
tx_hex = w3.to_hex(tx_hash)

print("↗ broadcast :", tx_hex)
print("⏳ waiting …")
print("✅ status   :", w3.eth.wait_for_transaction_receipt(tx_hash).status)  # 1 = success

↗ broadcast : 0x8b4b4e4d99d92b503cdaa754d6502cf82a3bae8fd37e6578c25eaa98831eb6e1
⏳ waiting …
✅ status   : 0


In [73]:
print ("https://eth.blockscout.com/tx/" + tx_hex)

https://eth.blockscout.com/tx/0x8b4b4e4d99d92b503cdaa754d6502cf82a3bae8fd37e6578c25eaa98831eb6e1
